In [4]:
from schemdraw import Drawing
from schemdraw import elements as elm
import tkinter as tk
from tkinter import ttk
from schemdraw.backends.mpl import Figure
import io
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

class CircuitApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Circuit Calculator")
        Drawing.backend = 'matplotlib'
        plt.ioff()  # Turn off interactive mode
        
        self.create_menu_bar()
        
        self.main_container = tk.PanedWindow(self.root, orient=tk.HORIZONTAL)
        self.main_container.pack(fill=tk.BOTH, expand=True)
        
        self.create_sidebar()
        self.create_canvas()
        self.setup_component_handlers()
        
        self.sidebar_visible = True
        self.current_component = None
        self.drawing = Drawing()
        self.temp_component = None
        self.mouse_x = 0
        self.mouse_y = 0
        
    def create_menu_bar(self):
        menubar = tk.Menu(self.root)
        self.root.config(menu=menubar)
        
        file_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="New")
        file_menu.add_command(label="Open")
        file_menu.add_command(label="Save")
        
        edit_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Edit", menu=edit_menu)
        
        view_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="View", menu=view_menu)
        
    def create_sidebar(self):
        self.sidebar_frame = tk.Frame(self.main_container, width=200, bg='white')
        self.main_container.add(self.sidebar_frame)
        
        search_frame = tk.Frame(self.sidebar_frame, bg='white')
        search_frame.pack(fill=tk.X, padx=5, pady=5)
        
        search_entry = tk.Entry(search_frame)
        search_entry.pack(fill=tk.X)
        search_entry.insert(0, "Search (press Q to jump here)")
        
        self.tree = ttk.Treeview(self.sidebar_frame, show='tree')
        self.tree.pack(fill=tk.BOTH, expand=True, padx=5)
        self.tree.bind('<Double-Button-1>', self.start_component_drag)
        
        categories = {
            "Favorites": [],
            "Recently Used": [],
            "Building Blocks": [],
            "Passive": ["Resistor", "Capacitor", "Inductor"],
            "Sources": ["Voltage Source", "AC Source", "Battery"],
            "Connectivity": ["Ground", "Wire"],
            "Semiconductors": [],
            "Electro-Mechanical": [],
            "Shapes": []
        }
        
        for category, items in categories.items():
            category_id = self.tree.insert("", "end", text=category)
            for item in items:
                self.tree.insert(category_id, "end", text=item)
                
    def create_canvas(self):
        self.canvas_frame = tk.Frame(self.main_container)
        self.main_container.add(self.canvas_frame)
        
        self.canvas = tk.Canvas(self.canvas_frame, bg='white')
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        self.draw_grid()
        
    def draw_grid(self):
        grid_size = 20
        canvas_width = 2000
        canvas_height = 2000
        
        for x in range(0, canvas_width, grid_size):
            self.canvas.create_line(x, 0, x, canvas_height, fill='#e0e0e0')
        for y in range(0, canvas_height, grid_size):
            self.canvas.create_line(0, y, canvas_width, y, fill='#e0e0e0')
            
    def setup_component_handlers(self):
        self.canvas.bind("<Button-1>", self.place_component)
        self.canvas.bind("<Motion>", self.update_component_position)
        self.component_functions = {
            "Resistor": self.add_resistor,
            "Capacitor": self.add_capacitor,
            "Inductor": self.add_inductor,
            "Voltage Source": self.add_voltage_source,
            "AC Source": self.add_ac_source,
            "Battery": self.add_battery
        }

    def start_component_drag(self, event):
        item = self.tree.identify('item', event.x, event.y)
        if item:
            component_name = self.tree.item(item)['text']
            if component_name in self.component_functions:
                self.current_component = component_name
                # Get initial mouse position relative to canvas
                canvas_x = self.canvas.winfo_rootx()
                canvas_y = self.canvas.winfo_rooty()
                mouse_x = self.root.winfo_pointerx() - canvas_x
                mouse_y = self.root.winfo_pointery() - canvas_y
                self.create_temp_component(mouse_x, mouse_y)

    def create_temp_component(self, x, y):
        if self.temp_component:
            self.canvas.delete(self.temp_component)
        
        d = Drawing()
        d.unit = 0.5  # Made components smaller
        component = {
            "Resistor": elm.Resistor(),
            "Capacitor": elm.Capacitor(),
            "Inductor": elm.Inductor2(),
            "Voltage Source": elm.SourceV(),
            "AC Source": elm.SourceSin(),
            "Battery": elm.Battery()
        }[self.current_component].at((0, 0))
        
        d += component
        
        # Create the figure without displaying it
        d.draw()  # This creates the matplotlib figure
        
        buffer = io.BytesIO()
        d.fig.canvas.print_png(buffer)  # Use print_png instead of savefig
        plt.close(d.fig)
        
        buffer.seek(0)
        image = Image.open(buffer)
        self.component_image = ImageTk.PhotoImage(image)
        self.temp_component = self.canvas.create_image(
            x, y, image=self.component_image, anchor='center'
        )
        buffer.close()

    def update_component_position(self, event):
        if self.current_component and self.temp_component:
            self.mouse_x = event.x
            self.mouse_y = event.y
            self.canvas.coords(self.temp_component, event.x, event.y)

    def place_component(self, event):
        if self.current_component and self.temp_component:
            x, y = event.x, event.y
            self.component_functions[self.current_component](x, y)
            self.canvas.delete(self.temp_component)
            self.temp_component = None
            self.current_component = None
            self.canvas.unbind("<Motion>")

    def add_resistor(self, x, y):
        d = Drawing()
        d.unit = 1
        d += elm.Resistor().at((x, y))
        self.draw_on_canvas(d)

    def add_capacitor(self, x, y):
        d = Drawing()
        d.unit = 1
        d += elm.Capacitor().at((x, y))
        self.draw_on_canvas(d)

    def add_inductor(self, x, y):
        d = Drawing()
        d.unit = 1
        d += elm.Inductor2().at((x, y))
        self.draw_on_canvas(d)

    def add_voltage_source(self, x, y):
        d = Drawing()
        d.unit = 1
        d += elm.SourceV().at((x, y))
        self.draw_on_canvas(d)

    def add_ac_source(self, x, y):
        d = Drawing()
        d.unit = 1
        d += elm.SourceSin().at((x, y))
        self.draw_on_canvas(d)

    def add_battery(self, x, y):
        d = Drawing()
        d.unit = 1
        d += elm.Battery().at((x, y))
        self.draw_on_canvas(d)

    def draw_on_canvas(self, drawing):
        drawing.draw()  # This creates the matplotlib figure
        
        buffer = io.BytesIO()
        drawing.fig.canvas.print_png(buffer)  # Use print_png instead of savefig
        plt.close(drawing.fig)
        
        buffer.seek(0)
        image = Image.open(buffer)
        photo_image = ImageTk.PhotoImage(image)
        self.canvas.image_references = getattr(self.canvas, 'image_references', []) + [photo_image]
        self.canvas.create_image(self.mouse_x, self.mouse_y, image=photo_image, anchor='center')
        buffer.close()

    def on_component_select(self, event):
        selected_item = self.tree.selection()[0]
        component_name = self.tree.item(selected_item)['text']
        if component_name in self.component_functions:
            self.current_component = component_name

    def run(self):
        self.root.geometry("1200x800")
        self.root.mainloop()

if __name__ == "__main__":
    app = CircuitApp()
    app.run()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\gsvolpato\AppData\Local\Temp\ipykernel_13952\4225772181.py", line 123, in start_component_drag
    self.create_temp_component(mouse_x, mouse_y)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "C:\Users\gsvolpato\AppData\Local\Temp\ipykernel_13952\4225772181.py", line 146, in create_temp_component
    d.fig.canvas.print_png(buffer)  # Use print_png instead of savefig
    ^^^^^^^^^^^^
AttributeError: 'Figure' object has no attribute 'canvas'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           

In [5]:
pip install skidl

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.1 MB ? eta -:--:--
     ------- -------------------------------- 0.8/4.1 MB 3.0 MB/s eta 0:00:02
     ------------ --------------------------- 1.3/4.1 MB 2.6 MB/s eta 0:00:02
     --------------- ------------------------ 1.6/4.1 MB 2.6 MB/s eta 0:00:01
     ----------------------- ---------------- 2.4/4.1 MB 2.7 MB/s eta 0:00:01
     ------------------------------ --------- 3.1/4.1 MB 2.7 MB/s eta 0:00:01
     ------------------------------------ --- 3.7/4.1 MB 2.8 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 2.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metada